# Data 

In [1]:
import pandas as pd 
distances = pd.read_stata("/Users/jeffreyohl/Downloads/bilateral_distances.dta")


# 1. Pivot to make each row a unique "res" and each column a unique "work"
pivoted = distances.pivot(
    index='state_county_res',
    columns='state_county_work',
    values='dist'
)

# 2. Sort the index and columns so that the smallest codes appear first
pivoted = pivoted.sort_index(axis=0).sort_index(axis=1)

# 3. Convert the pivoted DataFrame to a NumPy array
dist_ni = pivoted.to_numpy()

In [2]:
import pandas as pd 

flows = pd.read_stata("/Users/jeffreyohl/Documents/GitHub/Spatial/data/Monte et al 2018 Replication Folder/CMLEE.dta")

/var/folders/g9/ggrmf_5j6tx4rv5qdhs0slq40000gn/T/ipykernel_50756/3461067768.py:3: UnicodeWarning: 
One or more strings in the dta file could not be decoded using utf-8, and
so the fallback encoding of latin-1 is being used.  This can happen when a file
has been incorrectly encoded by Stata or some other software. You should verify
the string values returned are correct.
  flows = pd.read_stata("/Users/jeffreyohl/Documents/GitHub/Spatial/data/Monte et al 2018 Replication Folder/CMLEE.dta")


In [3]:
flows

,state_name_res,state_fips_res,state_name_work,state_fips_work,county_name_res,county_name_work,state_county_res,state_county_work,dist,workplace_emp,cz2000_res,cz2000_work,commuting,residence_emp,fr_outsideres,fr_outsidework,avgw_pow
0,Alabama,1,Alabama,1,Autauga County,Mobile County,1001,1097,252.818512,234725,60,254,0.000000,34240.335938,NaN,NaN,40311.371094
1,Alabama,1,Alabama,1,Autauga County,Pickens County,1001,1107,159.675552,6386,60,216,0.000000,34240.335938,NaN,NaN,31380.207031
2,Alabama,1,Oklahoma,40,Autauga County,Tulsa County,1001,40143,940.995728,445827,60,40,0.000000,34240.335938,NaN,NaN,51681.523438
3,Alabama,1,Alabama,1,Autauga County,Franklin County,1001,1059,239.109848,15082,60,140,0.000000,34240.335938,NaN,NaN,30511.404297
4,Alabama,1,Georgia,13,Autauga County,Rabun County,1001,13241,397.059875,8291,60,123,0.000000,34240.335938,NaN,NaN,29258.833984
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128723,Wyoming,56,Wyoming,56,Weston County,Crook County,56045,56011,82.572121,4169,445,445,75.924118,5695.854004,NaN,NaN,30836.890625
128724,Wyoming,56,Wyoming,56,Weston County,Niobrara County,56045,56027,87.513115,1741,445,666,15.663063,5695.854004,NaN,NaN,26029.292969
128725,Wyoming,56,Montana,30,Weston County,Big Horn County,56045,30003,290.620361,6368,445,595,0.000000,5695.854004,NaN,NaN,34935.300781
128726,Wyoming,56,Wyoming,56,Weston County,Campbell County,56045,56005,89.891357,32351,445,445,643.807800,5695.854004,NaN,NaN,58798.613281


In [4]:
pwd()

'/Users/jeffreyohl/Documents/GitHub/Spatial/code'

In [5]:
# do assure leading zeros are not included on fips, etc if we want to merge later. that's what the other ones do. 


import numpy as np

# Read space-delimited file, skipping header row

data = pd.read_csv('../data/Raw Data/CountyCentroids2010.txt', 
                delimiter='\t',
                encoding='latin-1')
data = (
    data[['GEOID', 'ALAND']]
    .drop_duplicates()                # remove duplicates based on entire row
    .sort_values(by='GEOID', ascending=True)
)

land_area = data[['ALAND', 'GEOID']]
land_area = np.array(land_area['ALAND'])

In [6]:
land_area

array([1539582278, 4117521611, 2291818968, ...,   92298550,  143005172,
        176615594])

In [7]:
residence_emp = (
    flows[['state_county_res', 'residence_emp']]
    .drop_duplicates()                # remove duplicates based on entire row
    .sort_values(by='state_county_res', ascending=True)
)

workplace_emp = (
    flows[['state_county_work', 'workplace_emp']]
    .drop_duplicates()                # remove duplicates based on entire row
    .sort_values(by='state_county_work', ascending=True)
)
avgw_pow = (
    flows[['state_county_work', 'avgw_pow']]
    .drop_duplicates()  # remove duplicate rows
    .sort_values(by='state_county_work')
)

# Fake data
workemp   =  np.array(workplace_emp['workplace_emp'])
workwage  = np.array(avgw_pow['avgw_pow'])
    

    
# Suppose R (residents), D, vbar are also known (random example):
R     = np.array(workplace_emp['workplace_emp']) 

In [8]:
# 1) Compute the pivot table using sum for duplicates (if needed) and fill with 0
df_pivot = flows.pivot_table(
    index='state_county_res',
    columns='state_county_work',
    values='commuting',
    aggfunc='sum',      # sum over duplicates
    fill_value=0
)

# 2) Sort rows and columns in ascending order
df_pivot = df_pivot.sort_index().sort_index(axis=1)

# 3) Divide by the sum of commuting over the entire DataFrame, so each cell becomes a share
total_commuting = flows['commuting'].sum()
df_pivot = df_pivot / total_commuting

df_pivot
lambda_ni = np.array(df_pivot)
lambda_nR = lambda_ni.sum(axis=1)
# Broadcast division over each row using [:, None] to match dimensions
lambda_ni_given_n = lambda_ni / lambda_nR[:, None]   # shape: (N, M)
vbar_n = (lambda_ni_given_n * workwage).sum(axis=1)  # shape: (N,)

# These are all the non-zero pairs

In [9]:
workplace_emp

,state_county_work,workplace_emp
13,1001,19141
106,1003,92474
88,1005,14328
14,1007,6685
174,1009,19296
...,...,...
3173,56037,30707
3216,56039,26940
23949,56041,13218
9954,56043,5639


In [10]:
residence_emp

,state_county_res,residence_emp
0,1001,34240.335938
35,1003,104695.351562
119,1005,13461.859375
154,1007,11974.728516
168,1009,34892.777344
...,...,...
128647,56037,30729.466797
128674,56039,23802.765625
128685,56041,13208.846680
128707,56043,5422.832031


In [11]:
avgw_pow

,state_county_work,avgw_pow
13,1001,33467.375000
106,1003,31443.085938
88,1005,32658.990234
14,1007,32004.636719
174,1009,25252.642578
...,...,...
3173,56037,55855.960938
3216,56039,40985.968750
23949,56041,43990.996094
9954,56043,35345.097656


In [ ]:
import numpy as np

def rhsProductivities(A, L, w, d, R, D, sigma, vbar):
    """
    Returns an array 'labIncomeHat' of shape (N,), where:
      labIncomeHat[i] = sum_n [
        L[i]*(d[n,i]*w[i]/A[i])^(1-sigma)
        / sum_k [ L[k]*(d[n,k]*w[k]/A[k])^(1-sigma) ]
      ] * [vbar[n]*R[n] + D[n]].

    Parameters
    ----------
    A : array of shape (N,)
        Current guess of A^(sigma-1) or A (depending on usage).
        Make sure A > 0.
    L, w : arrays of shape (N,)
        Employment and wages for each region i.
    d    : (N x N) array with d[n,i] = distance factor from n to i.
    R, D, vbar : arrays of shape (N,)
        R[n], D[n], vbar[n] for region n.
    sigma : float
        Elasticity (> 1).
    
    Returns
    -------
    labIncomeHat : array of shape (N,)
        Model-implied labor income for each i, i.e. the sum over n
        of fraction_{n,i} * (vbar[n]*R[n] + D[n]).
    """
    N = len(A)
    # M[n,i] = L[i]*(d[n,i]*w[i]/A[i])^(1-sigma)
    M = L * np.power(d * (w / A), 1.0 - sigma)  # shape (N, N), numerator
    
    # denom[n] = sum_k M[n,k]
    denom = np.sum(M, axis=1)                   # shape (N,)
    
    # X[n] = vbar[n]*R[n] + D[n]
    X = vbar * R + D                            # shape (N,)
    
    # fraction_{n,i} = M[n,i] / denom[n].
    # partial_term[i,n] = fraction_{n,i} * X[n] => we use transpose.
    M_T = M.T  # shape (N, N), M_T[i,n] = M[n,i]
    summand = (M_T / denom) * X  # broadcast X[n], denom[n]
    labIncomeHat = np.sum(summand, axis=1) # sum over n => shape (N,)
    
    return labIncomeHat


def productivities(
    workemp,      # L_i
    workwage,     # w_i
    resExp,       # dictionary or struct with R, D, vbar
    distmatrix,   # NxN
    sigma,        # elasticity
    psi,          # distance exponent
    detailsYN=True,
    max_iter=1000,
    tol=1e-6
):
    """
    Python version of the Mathematica-style tâtonnement for solving
      w_i L_i == sum_n fraction_{n,i} * [vbar[n]*R[n] + D[n]],
    where fraction_{n,i} = M[n,i]/sum_k M[n,k], M[n,i]=(L_i*(d_{n,i}*w_i/A_i)^(1-sigma)).
    
    We iterate on A^(sigma-1) until the implied w_i L_i matches the actual w_i L_i.
    
    Returns
    -------
    A_final : array of shape (N,) = A_i^(1/(sigma-1)), i.e. the final productivities.
    """

    ncounties = len(workemp)
    
    #------------------------------------------------
    # 1) Preprocess / scaling as per your code
    #    You can adjust these scalings to match your
    #    original Mathematica approach exactly.
    #------------------------------------------------
    
    
    L = workemp  # employees in hundreds
    w = workwage   #wage in tens of thousands thousands

    # Let d = distmatrix^(psi)
    d = distmatrix**psi

    # Unpack R, D, vbar from resExp dict (adjust as needed)
    R = resExp["R"]       # array of shape (N,)
    D_ = resExp["D"]      # array of shape (N,)
    vbar = resExp["vbar"] # array of shape (N,)

    # We'll compute actual "labIncome" = w_i L_i, but note
    # we used w=workwage/1e4 => actual w_i L_i is
    #    (workwage[i]/1e4)*workemp[i] ...
    # If your code wants it in "millions" => divide by 1e6
    # or do whichever scaling you used in Mathematica:
    labIncome = (L * w)  # in
    
    # Step size (partial adjustment)
    lam = 0.990
    
    #------------------------------------------------
    # 2) Initialize A0 as A^(sigma-1) = 1
    #------------------------------------------------
    A0 = np.ones(ncounties)
    
    #------------------------------------------------
    # 3) Iteration
    #------------------------------------------------
    error = 1.0  # track max gap
    c = 0
    
    while error > tol and c < max_iter:
        c += 1
        
        # Model-implied income under current guess A0
        # Note we pass D_ (because "D" is a python builtin),
        # but just rename it for clarity.
        labIncomeHat = rhsProductivities(A0, L, w, d, R, D_, sigma, vbar)
        print("lab income")
        print(labIncome)
        print("total labor income true")
        print(np.sum(labIncome))
        print("lab income hat")
        print(labIncomeHat)
        print("total lab income hat")
        print(np.sum(labIncomeHat))
        # Compare to actual labIncome => gap[i] = actual[i]/hat[i]
        gap = labIncome / labIncomeHat
       
        # "dist" is the maximum relative difference from 1
        error = np.max(np.abs(gap - 1.0))
        distmin = np.min(np.abs(gap - 1.0))
        
        # Partial update: A1 = [ lam + (1-lam)*gap ] * A0
        A1 = (lam + (1.0 - lam)*gap) * A0
        
        # Rescale so that mean(A1) = 1 (like your code)
        A1_mean = np.mean(A1) 
        A0 = A1 / A1_mean # normalize to mean 1.
        print("mean A0")
        print(np.mean(A0))
        print("A0")
        print(A0)
        # Optional: print diagnostics
        if detailsYN and c % 50 == 1 and error > 0.01:
            print(f"Iteration {c}, max gap={error:8.5f}, min gap={distmin:8.5f}, "
                  f"A0 in [{A0.min():.3e}, {A0.max():.3e}]")
            

    if detailsYN:
        print(f"End of iteration {c}, max gap={error:8.5f}, "
              f"A0 in [{A0.min():.3e}, {A0.max():.3e}]")
    
    #------------------------------------------------
    # 4) Exponentiate back to get true A
    #    A^(1/(sigma-1)) and rescale so mean(...)=1
    #------------------------------------------------
    exponent = 1.0 / (sigma - 1.0)
    A_linear = A0**exponent
    A_linear /= np.mean(A_linear)
    
    return A_linear


# ------------------------------------------------------------------------
# EXAMPLE USAGE (dummy example)
if __name__ == "__main__":
    np.random.seed(0)
    ncounties = len(workplace_emp)
    


    total_vbar_R = np.sum(vbar_n * R)  # scalar
    total_wage_emp = np.sum(workwage * workemp)  # scalar

    difference = total_vbar_R - total_wage_emp
    # allocate deficits equally. 
    D     = np.zeros(ncounties) - (difference)/ ncounties     # no deficits -- need from Yulia 
    # Put them in resExp
    resExp = {"R": R, "D": D, "vbar": vbar_n}
    
    sigma = 4.0
    psi   = 0.43 #need from Yulia
    
    A_solution = productivities(
        workemp=workemp,
        workwage=workwage,
        resExp=resExp,
        distmatrix=dist_ni,
        sigma=sigma,
        psi=psi,
        detailsYN=True,
        max_iter=1000,
        tol=1e-8
    )
    print("\nFinal productivities A:", A_solution)


lab income
[6.40599025e+08 2.90766793e+09 4.67938012e+08 ... 5.81472986e+08
 1.99311006e+08 1.54419001e+08]
total labor income true
8872296980627.219
lab income hat
[1.27946737e+09 7.13857045e+09 9.74790543e+08 ... 3.08980472e+08
 2.03165586e+08 2.98705647e+08]
total lab income hat
8872296980627.219
mean A0
1.0
A0
[0.99643762 0.99550269 0.99623095 ... 1.0102698  1.00124804 0.99660069]
Iteration 1, max gap=204.95542, min gap= 0.00046, A0 in [9.915e-01, 3.054e+00]
lab income
[6.40599025e+08 2.90766793e+09 4.67938012e+08 ... 5.81472986e+08
 1.99311006e+08 1.54419001e+08]
total labor income true
8872296980627.219
lab income hat
[1.26298001e+09 7.02623450e+09 9.61617293e+08 ... 3.17062183e+08
 2.02952676e+08 2.94330838e+08]
total lab income hat
8872296980627.219
mean A0
1.0
A0
[0.99374758 0.99188348 0.99333582 ... 1.02097597 1.00331003 0.99408433]
lab income
[6.40599025e+08 2.90766793e+09 4.67938012e+08 ... 5.81472986e+08
 1.99311006e+08 1.54419001e+08]
total labor income true
8872296980627

In [17]:
import numpy as np
output_folder_name = "../output/"
prod_path_name = output_folder_name + "productivities.csv"

np.savetxt(prod_path_name, A_solution, delimiter=',')


# Estimating Equation for $\phi$

In [42]:
import numpy as np
import pandas as pd
from linearmodels.iv import IV2SLS
from statsmodels.regression.linear_model import OLS
import statsmodels.api as sm
# Suppose you already have:
#   lambda_ni: 2D array of shape (N, I)
#   workwage:  1D array of shape (I,) for w_i
#   distm:     2D array of shape (N, I) for dist_{ni}
#   A_solution:1D array of shape (I,) (the excluded instrument)
#   N = number of residence areas, I = number of workplaces

# 1) Build a long-format DataFrame, dropping rows where lambda_ni == 0
rows = []
N, I = lambda_ni.shape
for n in range(N):
   
    for i in range(I):
        if lambda_ni[n, i] > 0:
            rows.append({
                'n': n,  # residence index
                'i': i,  # workplace index
                'log_lambda': np.log(lambda_ni[n, i]),
                'log_wage':   np.log(workwage[i]),
                'log_dist':   np.log(dist_ni[n, i]),
                'log_A_inst':     np.log(A_solution[i])
            })
df = pd.DataFrame(rows)
def demean_two_way(df, y_col, x_cols, id1_col, id2_col, max_iter=100, tol=1e-8):
    """
    Demean variables for two-way fixed effects using iterative method.
    """
    y = df[y_col].copy()
    X = df[x_cols].copy()
    
    # Initialize
    y_prev = y.copy() + 1
    iter_count = 0
    
    while (np.abs(y - y_prev) > tol).any() and iter_count < max_iter:
        y_prev = y.copy()
        
        # Demean with respect to id1
        y = y - y.groupby(df[id1_col]).transform('mean')
        X = X - X.groupby(df[id1_col]).transform('mean')
        
        # Demean with respect to id2
        y = y - y.groupby(df[id2_col]).transform('mean')
        X = X - X.groupby(df[id2_col]).transform('mean')
        
        iter_count += 1
    
    return y, X
y_demean, X_demean = demean_two_way(
    df,
    y_col='log_lambda',
    x_cols=['log_dist'],
    id1_col='n',
    id2_col='i'
)

# Run regression on demeaned variables
X_demean = sm.add_constant(X_demean)  # Add constant
model = sm.OLS(y_demean, X_demean)
results = model.fit()



In [46]:
phi = -results.params['log_dist']
phi

np.float64(3.9783362029140705)

# Estimating Equation for $\epsilon$

In [47]:
df['y'] = df['log_lambda'] + 3.9783*df['log_dist']

# 3) Sweep out fixed effects by demeaning over n. (not sure how sketchy this is)
#    For each residence n, subtract the mean (over i) from every variable.
df['y_demeaned'] = df['y'] - df.groupby('n')['y'].transform('mean')
df['log_wage_demeaned'] = df['log_wage'] - df.groupby('n')['log_wage'].transform('mean')
df['log_A_inst_demeaned'] = df['log_A_inst'] - df.groupby('n')['log_A_inst'].transform('mean')

# 4) Run the 2SLS regression (no constant, since we've already demeaned).
#    y_demeaned = epsilon * log_wage_demeaned + error
#    with log_wage_demeaned instrumented by A_inst_demeaned.
formula = 'y_demeaned ~ 0 + [log_wage_demeaned ~ log_A_inst_demeaned]'
mod = IV2SLS.from_formula(formula, data=df)
res = mod.fit(cov_type='robust')  # Or choose your preferred cov_type
res

IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:             y_demeaned   R-squared:                      0.2229
Estimator:                    IV-2SLS   Adj. R-squared:                 0.2229
No. Observations:               45058   F-statistic:                    8900.2
Date:                Fri, Jan 31 2025   P-value (F-stat)                0.0000
Time:                        12:19:51   Distribution:                  chi2(1)
Cov. Estimator:                robust                                         
                                                                              
                                 Parameter Estimates                                 
=====================================================================================
                   Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
-------------------------------------------------------------------------------------
log_wage_demeaned     3.1056     0.0329     94.341     0.0000      3.0410      3.1701
=====================================================================================

Endogenous: log_wage_demeaned
Instruments: log_A_inst_demeaned
Robust Covariance (Heteroskedastic)
Debiased: False
IVResults, id: 0x16833b5c0

In [48]:
epsilon = res.params['log_wage_demeaned']
epsilon

np.float64(3.1055552730724476)

In [49]:
import pandas as pd

# Create a DataFrame with the parameters
parameters = pd.DataFrame({
    'Parameter': ['epsilon', 'phi'],
    'Value': [epsilon, phi]
})

# Define the output file path
output_file_path = output_folder_name + "commuting_parameters.csv"

# Save the DataFrame to a CSV file
parameters.to_csv(output_file_path, index=False)

In [ ]:
import requests
import pandas as pd

API_KEY = "488b0df48f856b207eae540095aee8cd37926d3a"

# Base URL for the 2006-2010 ACS 5-year "profile" data
base_url = "https://api.census.gov/data/2010/acs5/profile"

# We'll request the county-level percent of adults (25+) with bachelor's or higher
# DP02_0067PE = Percent with a bachelor's degree or higher (ACS 5-year profile)
# NAME gives the county name
params = {
    "get": "NAME,DP02_0067PE",
    "for": "county:*",
    "key": API_KEY
}

response = requests.get(base_url, params=params)
if response.status_code != 200:
    raise Exception(f"Request failed: {response.text}")

data = response.json()

# The first row of 'data' is the header, subsequent rows are the actual data
header = data[0]
rows = data[1:]

# Create a DataFrame from the JSON data
college_data = pd.DataFrame(rows, columns=header)

# Rename columns for clarity
college_data.rename(
    columns={
        "NAME": "County_Name",
        "DP02_0067PE": "Pct_Bachelors_Or_Higher",
        "state": "State_FIPS",
        "county": "County_FIPS"
    },
    inplace=True
)

# Create a combined FIPS code
college_data["FIPS"] = college_data["State_FIPS"] + college_data["County_FIPS"]

# Convert percentage to numeric
college_data["Pct_Bachelors_Or_Higher"] = pd.to_numeric(college_data["Pct_Bachelors_Or_Higher"], errors="coerce")

print(college_data.head())

# If you want, you can save the DataFrame to a CSV
college_data.to_csv("../data/Raw Data/acs_2006_2010_bachelors_or_higher_by_county.csv", index=False)
